In [ ]:
import json
import os
import pandas as pd
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

In [ ]:
prod_research_sql_credentials = json.load(open(os.environ['PROD_RESEARCH_WRITE_SQL_CREDENTIALS']))
rds_access_utils = RDSAccessUtils(prod_research_sql_credentials)
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
query = 'select * from keypoint_annotations where pen_id=48;'
df = rds_access_utils.extract_from_database(query)

In [ ]:
m = MetaData(schema='research')
Base = automap_base(metadata=m)
Base.prepare(rds_access_utils.sql_engine, reflect=True)
FishMetadata = Base.classes.fish_metadata


In [ ]:
Session = sessionmaker(bind=rds_access_utils.sql_engine)
session = Session()

In [ ]:
left_image_url

In [ ]:
research_sql_credentials = json.load(open(os.environ['SQL_CREDENTIALS']))
research_rds_access_utils = RDSAccessUtils(research_sql_credentials)
ResearchBase = automap_base()
ResearchBase.prepare(research_rds_access_utils.sql_engine, reflect=True)
GtsfDataCollection = ResearchBase.classes.gtsf_data_collections

In [ ]:
ResearchSession = sessionmaker(bind=research_rds_access_utils.sql_engine)
research_session = ResearchSession()

In [ ]:
count = 0
for idx, row in df.iterrows():
    if count % 100 == 0:
        print(count)
    count += 1
    left_image_url = row.left_image_url
    fish_metadatas = session.query(FishMetadata).filter(FishMetadata.left_url == left_image_url).all()
    if len(fish_metadatas) > 0:
        continue
    image_key = left_image_url[left_image_url.index('aquabyte-crops'):].replace('aquabyte-crops/', '')
    capture_bucket = 'aquabyte-frames-resized-inbound'
    capture_dir = os.path.dirname(image_key)
    capture_key = os.path.join(capture_dir, 'capture.json')
    
    capture_f = s3_access_utils.download_from_s3(capture_bucket, capture_key)
    capture_metadata = json.load(open(capture_f))
    gtsf_fish_identifier = capture_metadata['gtsf_fish_identifier']
    
    gtsf_data_collection = \
        research_session.query(GtsfDataCollection).filter(GtsfDataCollection.gtsf_fish_identifier == gtsf_fish_identifier).all()[0]
    
    ground_truth_metadata = json.loads(gtsf_data_collection.ground_truth_metadata)
    metadata = None
    try:
        metadata = {
            'breadthMms': ground_truth_metadata['data']['breath'],
            'widdthMms': ground_truth_metadata['data']['width'],
            'lengthMms': ground_truth_metadata['data']['length'],
            'weightKgs': ground_truth_metadata['data']['weight'],
            'location': 'Generic',
            'penNumber': '0',
            'species': ground_truth_metadata['data']['species']
        }
    except:
        continue
        
    fish_metadata = FishMetadata(
        left_url=left_image_url,
        fish_id=gtsf_fish_identifier,
        weight=metadata['weightKgs'],
        data=metadata,
        stereo_parameters_url='https://aquabyte-stereo-parameters.s3-eu-west-1.amazonaws.com/L40013178_R40014310/latest_L40013178_R40014310_stereo-parameters.json',
    )
    
    session.add(fish_metadata)
    session.commit()
    
    
    
    

In [ ]:
research_session.rollback()

In [ ]:
metadata = {
    'breadthMms': ground_truth_metadata['data']['breath'],
    'widdthMms': ground_truth_metadata['data']['width'],
    'lengthMms': ground_truth_metadata['data']['length'],
    'weightKgs': ground_truth_metadata['data']['weight'],
    'location': 'Generic',
    'penNumber': '0',
    'species': ground_truth_metadata['data']['species']
}

In [ ]:
metadata

In [ ]:
(5.758-5.3)/5.3

In [ ]:
(5.429-5.3)/5.3